In [20]:
# Đọc file CSV
# Chạy các file để sử dụng mô hình hồi quy tuyến tính

# DonHang (Cái này nháp. Tui chạy tìm sp đi kèm thui -- Tui làm lộn :v)

# Chủ yếu chạy các file csv này --> Rồi sẽ in ra các file csv

# File csv gốc --> File csv sau khi dự đoán giá
# ChiPhiSPDuKien --> gia_ban_du_kien.csv
# GiaBanCua1SoCuaHang --> gia_ban_du_kien_tu_CH_canh_tranh.csv
# DuLieuBanHang --> gia_ban_du_kien_sau_4_thang.csv
# KyNghi --> khuyen_mai_ngay_le.csv

# --> Từ đó lấy các file csv được tạo ra
# --> Hồi quy tuyến tính lần nữa để đưa ra mức giá phù hợp nhất

================== Tìm SP đi kèm ================

In [ ]:
# 1. ====================== Bằng apriori tìm ra các tập phổ biến  =====================

# Từ file DonHang.csv

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# Đọc dữ liệu từ file CSV
df = pd.read_csv('DonHang.csv')

# Hiển thị số đơn hàng
print(f"Số đơn hàng: {df['MaDonHang'].nunique()}")

# Tạo ma trận đơn hàng - sản phẩm (1 nếu có mua, 0 nếu không)
basket = df.groupby(['MaDonHang', 'Ma SP'])['SoLuong'].sum().unstack().fillna(0)
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

# Áp dụng thuật toán Apriori để tìm các tập phổ biến
frequent_itemsets = apriori(basket, min_support=0.1, use_colnames=True)

# Kiểm tra nếu không tìm thấy tập phổ biến nào
if frequent_itemsets.empty:
    print("❌ Không tìm thấy tập phổ biến nào với min_support đã chọn.")
else:
    print("\n✅ Các tập phổ biến:")
    print(frequent_itemsets)

    # Tạo luật kết hợp
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

    print("\n✅ Các luật kết hợp:")
    print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

    # === PHẦN LỌC THEO SẢN PHẨM ===
    sp_target = input("\n🔍 Nhập mã sản phẩm cần tìm (ví dụ: SP01): ").strip()

    # Lọc tập phổ biến chứa sản phẩm
    related_itemsets = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: sp_target in x)]
    print(f"\n📦 Các tập phổ biến có chứa '{sp_target}':")
    print(related_itemsets)

    # Lọc luật kết hợp liên quan đến sản phẩm
    related_rules = rules[
        rules['antecedents'].apply(lambda x: sp_target in x) |
        rules['consequents'].apply(lambda x: sp_target in x)
    ]

    print(f"\n🔗 Các luật kết hợp liên quan đến '{sp_target}':")
    print(related_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from collections import defaultdict

# Bước 1: Đọc dữ liệu từ file CSV
df = pd.read_csv('DonHang.csv')

# Bước 2: Gom nhóm sản phẩm theo đơn hàng thành danh sách giao dịch
donhang_dict = defaultdict(list)
for _, row in df.iterrows():
    donhang_dict[row['MaDonHang']].append(row['Ma SP'])

transactions = list(donhang_dict.values())

print("🛒 Danh sách các giao dịch:")
for t in transactions:
    print(t)

# Bước 3: Mã hóa dữ liệu giao dịch
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

# Bước 4: Áp dụng thuật toán Apriori
frequent_itemsets = apriori(df_encoded, min_support=0.1, use_colnames=True)

# Bước 5: In ra các tập phổ biến
print("\n✅ Các tập phổ biến tìm được:")
print(frequent_itemsets)

# Bước 6: Lọc theo MASP (mã sản phẩm cụ thể)
masp = input("\n🔍 Nhập mã sản phẩm cần tìm (ví dụ: SP01): ").strip()
related_itemsets = frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: masp in x)]

print(f"\n📦 Các tập phổ biến có liên quan đến '{masp}':")
print(related_itemsets)

# Bước 7: Sinh luật kết hợp và lọc theo MASP
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

related_rules = rules[
    rules['antecedents'].apply(lambda x: masp in x) |
    rules['consequents'].apply(lambda x: masp in x)
]

print(f"\n🔗 Các luật kết hợp liên quan đến '{masp}':")
print(related_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


 =============== Dự đoán giá ==================

In [ ]:
# 2. ================= Dữ đoán giá bán dự kiến bằng mô hình hồi quy tuyến tính ==================

# sử dụng mô hình hồi quy tuyến tính để dự đoán "Giá Bán Dự Kiến" cho từng sản phẩm có trong file CSV

# Đọc dữ liệu từ file ChiPhiSPDuKien_csv.csv

In [ ]:
1. Đọc dữ liệu: Đọc dữ liệu từ file ChiPhiSPDuKien_csv.csv.
2. Xử lý dữ liệu:
3. Đổi tên cột 'Khuyen Mai(%)' thành 'Khuyen Mai (%)' và 'Danh Gia(%)' thành 'Danh Gia (%)'.
4. Chọn các cột chi phí, khuyến mãi, đánh giá làm features và 'Gia Ban Du Kien' làm target.
5. Loại bỏ các hàng có giá trị không phải số trong các cột features và target.
6. Loại bỏ các hàng có giá trị NaN sau khi chuyển đổi kiểu số.
7. Chia dữ liệu: Chia dữ liệu thành tập huấn luyện (80%) và tập kiểm tra (20%).
8. Huấn luyện mô hình: Huấn luyện mô hình hồi quy tuyến tính trên tập huấn luyện.
9. Dự đoán: Dự đoán 'Gia Ban Du Kien' cho toàn bộ dữ liệu sử dụng mô hình đã huấn luyện.
10. In kết quả: In ra DataFrame chứa 'Ma SP' và giá dự đoán.
11. Đánh giá mô hình: Tính toán và in ra các chỉ số đánh giá MSE, R-squared và MAE trên tập kiểm tra.
12. Xuất file CSV: Lưu DataFrame kết quả dự đoán vào file gia_ban_du_kien.csv.

In [73]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def xuat_file_csv(df, ten_file, index=False, encoding='utf-8'):
    """
    Hàm xuất DataFrame ra file CSV.

    Args:
        df (pd.DataFrame): DataFrame cần xuất.
        ten_file (str): Tên file CSV (ví dụ: 'output.csv').
        index (bool, optional): Có ghi index của DataFrame vào file CSV không. Mặc định là False.
        encoding (str, optional): Mã hóa của file CSV. Mặc định là 'utf-8'.
    """
    try:
        df.to_csv(ten_file, index=index, encoding=encoding)
        print(f"Đã xuất thành công file CSV: '{ten_file}'")
    except Exception as e:
        print(f"Đã xảy ra lỗi khi xuất file CSV: {e}")

# Đọc dữ liệu từ file CSV
try:
    df_input = pd.read_csv('ChiPhiSPDuKien_csv.csv', encoding='utf-8')
except FileNotFoundError:
    print("Không tìm thấy file ChiPhiSPDuKien_csv.csv. Vui lòng kiểm tra lại đường dẫn.")
    exit()

# Đổi tên cột cho khớp (nếu cần)
df_input.rename(columns={'Khuyen Mai(%)': 'Khuyen Mai (%)', 'Danh Gia(%)': 'Danh Gia (%)'}, inplace=True)

# Chọn các cột làm biến độc lập (features) và biến mục tiêu (target)
features = ['Chi Phi Nguyen Lieu', 'Chi Phi Nhan Cong', 'Chi Phi Van Hanh', 'Tong Chi Phi', 'Khuyen Mai (%)', 'Danh Gia (%)']
target = 'Gia Ban Du Kien'

# Loại bỏ các hàng có giá trị không phải số trong các cột features
for col in features:
    df_input = df_input[pd.to_numeric(df_input[col], errors='coerce').notna()]

# Chuyển đổi cột target sang kiểu số
df_input = df_input[pd.to_numeric(df_input[target], errors='coerce').notna()]
df_input[target] = pd.to_numeric(df_input[target], errors='coerce')

# Loại bỏ các hàng có giá trị NaN sau khi chuyển đổi
df_input.dropna(subset=features + [target], inplace=True)

X = df_input[features]
y = df_input[target]

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Khởi tạo và huấn luyện mô hình hồi quy tuyến tính
model = LinearRegression()
model.fit(X_train, y_train)

# Dự đoán giá bán dự kiến cho TẤT CẢ sản phẩm trong file
y_pred_all = model.predict(X)

# Tạo một DataFrame mới chứa Ma SP và Giá Bán Dự Kiến đã dự đoán
df_predicted = pd.DataFrame({'MaSP': df_input['Ma SP'], 'Gia Ban Du Kien': y_pred_all})

# In ra DataFrame chứa Ma SP và giá dự đoán
print("Giá bán dự kiến dự đoán cho từng sản phẩm:")
print(df_predicted)

# Đánh giá mô hình trên tập kiểm tra
y_pred_test = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred_test)
print(f'Mean Squared Error trên tập kiểm tra: {mse:.2f}\n')

print("\nĐánh giá mô hình:")
print(f"Mean Squared Error (MSE) trên tập kiểm tra: {mse:.2f}")

# Để có thêm các chỉ số đánh giá, bạn có thể import thêm các hàm từ sklearn.metrics
from sklearn.metrics import r2_score, mean_absolute_error

r2 = r2_score(y_test, y_pred_test)
mae = mean_absolute_error(y_test, y_pred_test)

print(f"R-squared (R^2) trên tập kiểm tra: {r2:.2f}")
print(f"Mean Absolute Error (MAE) trên tập kiểm tra: {mae:.2f}")

# Sử dụng hàm xuat_file_csv để xuất DataFrame kết quả ra file CSV
ten_file_xuat = 'gia_ban_du_kien.csv'
xuat_file_csv(df_predicted, ten_file_xuat)

Giá bán dự kiến dự đoán cho từng sản phẩm:
    MaSP  Gia Ban Du Kien
0   SP01             22.0
1   SP02             17.0
2   SP03             25.0
3   SP04             21.0
4   SP05             25.0
5   SP06             22.0
6   SP07             23.0
7   SP08             12.0
8   SP09             14.0
9   SP10             12.0
10  SP11             15.0
11  SP12             13.0
12  SP13             17.0
13  SP14             19.0
14  SP15             17.0
15  SP16             19.0
16  SP17             17.0
17  SP18             21.0
Mean Squared Error trên tập kiểm tra: 0.00


Đánh giá mô hình:
Mean Squared Error (MSE) trên tập kiểm tra: 0.00
R-squared (R^2) trên tập kiểm tra: 1.00
Mean Absolute Error (MAE) trên tập kiểm tra: 0.00
Đã xuất thành công file CSV: 'gia_ban_du_kien.csv'


In [ ]:
# ================= Giá bán của 1 số cửa hàng ĐỐI THỦ CẠNH TRANH ================

# đọc sữ liệu từ file GiaBanCua1SoCuaHang_csv.csv

# sử dụng mô hình hồi quy tuyến tính để dự đoán "Giá Bán" cho từng sản phẩm có trong file CSV


In [ ]:
1. Đọc dữ liệu: Đọc dữ liệu từ file GiaBanCua1SoCuaHang_csv.csv.
2. Xử lý dữ liệu:
3. Thay thế giá trị 'k' bằng NaN trong cột 'GiaBanCuaHang 20'.
4. Chuyển đổi các cột giá sang kiểu số và loại bỏ các hàng chứa NaN.
5. Chọn features và target: Chọn 'GiaBanCuaHang 1' làm target và các cột giá còn lại làm features.
6. Chia dữ liệu: Chia dữ liệu thành tập huấn luyện và tập kiểm tra.
7. Huấn luyện mô hình: Huấn luyện mô hình hồi quy tuyến tính.
8. Dự đoán: Dự đoán giá bán phù hợp cho tất cả các sản phẩm đã làm sạch dựa trên mô hình.
9. Đánh giá mô hình: Tính toán MSE, R-squared và MAE trên tập kiểm tra và đưa ra một đánh giá sơ bộ.
10. Xuất file CSV: Lưu kết quả dự đoán vào file gia_ban_du_kien_tu_CH_canh_tranh.csv

In [74]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

def xuat_file_csv(df, ten_file, index=False, encoding='utf-8'):
    """
    Hàm xuất DataFrame ra file CSV.

    Args:
        df (pd.DataFrame): DataFrame cần xuất.
        ten_file (str): Tên file CSV (ví dụ: 'output.csv').
        index (bool, optional): Có ghi index của DataFrame vào file CSV không. Mặc định là False.
        encoding (str, optional): Mã hóa của file CSV. Mặc định là 'utf-8'.
    """
    try:
        df.to_csv(ten_file, index=index, encoding=encoding)
        print(f"Đã xuất thành công file CSV: '{ten_file}'")
    except Exception as e:
        print(f"Đã xảy ra lỗi khi xuất file CSV: {e}")

# Tên file CSV
file_path = 'GiaBanCua1SoCuaHang_csv.csv'

try:
    # Đọc dữ liệu từ file CSV với dấu phân cách là dấu phẩy
    df = pd.read_csv(file_path, sep=',')

    # Thay thế giá trị 'k' trong cột 'GiaBanCuaHang 20' bằng NaN
    df['GiaBanCuaHang 20'] = df['GiaBanCuaHang 20'].replace('k', pd.NA)

    # Chuyển đổi tất cả các cột giá sang kiểu số (ép buộc lỗi thành NaN)
    gia_cua_hang_cols = [col for col in df.columns if col != 'MaSP']
    for col in gia_cua_hang_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    df_cleaned = df.dropna()

    if not df_cleaned.empty and len(gia_cua_hang_cols) > 1:
        # Chọn cột mục tiêu và các features
        target_col = 'GiaBanCuaHang 1'
        feature_cols = [col for col in gia_cua_hang_cols if col != target_col]

        X = df_cleaned[feature_cols]
        y = df_cleaned[target_col]

        # Chia dữ liệu thành tập huấn luyện và kiểm tra
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Huấn luyện mô hình hồi quy tuyến tính
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Dự đoán "Giá Bán" phù hợp dựa trên giá của tất cả các đối thủ
        # Chúng ta sẽ sử dụng toàn bộ dữ liệu đã làm sạch để dự đoán
        X_all = df_cleaned[feature_cols]
        predicted_gia_ban = model.predict(X_all)

        df_gia_ban_du_kien = pd.DataFrame({'MaSP': df_cleaned['MaSP'], 'GiaBanPhuHop': predicted_gia_ban})
        print("\nGiá Bán Phù Hợp Dự Kiến (dựa trên mô hình hồi quy):")
        print(df_gia_ban_du_kien)

        # Đánh giá mô hình (trên tập kiểm tra)
        y_pred_test = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred_test)
        r2 = r2_score(y_test, y_pred_test)
        mae = mean_absolute_error(y_test, y_pred_test)

        print("\nĐánh giá mô hình trên tập kiểm tra:")
        print(f'Mean Squared Error (MSE): {mse:.2f}')
        print(f'R-squared (R^2): {r2:.2f}')
        print(f'Mean Absolute Error (MAE): {mae:.2f}')

        # Đánh giá tốt/xấu (mang tính tương đối)
        print("\nĐánh giá mô hình:")
        if r2 > 0.7:
            print("Mô hình có vẻ phù hợp tốt với dữ liệu (R-squared cao).")
        elif r2 > 0.5:
            print("Mô hình ở mức chấp nhận được.")
        else:
            print("Mô hình có thể chưa phù hợp tốt.")

        if mae < (y_test.max() - y_test.min()) / 10:  # Ước tính một ngưỡng tương đối
            print("Sai số trung bình tuyệt đối (MAE) tương đối nhỏ.")
        else:
            print("Sai số trung bình tuyệt đối (MAE) có thể cần được cải thiện.")

        # Xuất DataFrame df_gia_ban_du_kien ra file CSV
        ten_file_xuat = 'gia_ban_du_kien_tu_CH_canh_tranh.csv'
        xuat_file_csv(df_gia_ban_du_kien, ten_file_xuat)

    else:
        print("\nKhông đủ dữ liệu hoặc cột để thực hiện hồi quy.")

except FileNotFoundError:
    print(f"Không tìm thấy file: {file_path}. Vui lòng kiểm tra lại tên file và đường dẫn.")
except Exception as e:
    print(f"Đã xảy ra lỗi: {e}")


Giá Bán Phù Hợp Dự Kiến (dựa trên mô hình hồi quy):
    MaSP  GiaBanPhuHop
0   SP01     21.912284
1   SP02     16.986384
2   SP03     27.000000
3   SP04     35.000000
4   SP05     23.000000
5   SP06     25.561616
6   SP07     25.000000
7   SP08     17.000000
8   SP09     17.422507
9   SP10     16.000000
10  SP11     17.000000
11  SP12     14.000000
12  SP13     22.000000
13  SP14     18.000000
14  SP15     20.000000
15  SP16     21.000000
16  SP17     20.000000
17  SP18     25.000000

Đánh giá mô hình trên tập kiểm tra:
Mean Squared Error (MSE): 2.48
R-squared (R^2): 0.81
Mean Absolute Error (MAE): 1.45

Đánh giá mô hình:
Mô hình có vẻ phù hợp tốt với dữ liệu (R-squared cao).
Sai số trung bình tuyệt đối (MAE) có thể cần được cải thiện.
Đã xuất thành công file CSV: 'gia_ban_du_kien_tu_CH_canh_tranh.csv'


In [ ]:
# ================= Số lượng khách mua bán sản phẩm trong 4 tháng ================

# đọc sữ liệu từ file DuLieuBanHang_csv.csv

# sử dụng mô hình hồi quy tuyến tính để dự đoán "Giá Bán" cho từng sản phẩm có trong file CSV


In [ ]:
1. Đọc dữ liệu: Đọc dữ liệu từ file DuLieuBanHang_csv.csv.
2. Đổi tên cột: Đổi tên cột 'SanPham' thành 'MaSP' và 'SoLuongBan' thành 'SoLuongBan'.
3. Chọn features và target: Chọn 'SoLuongBan' làm feature duy nhất và 'GiaBan' làm target.
4. Dự đoán theo từng sản phẩm:
5. Lặp qua từng sản phẩm duy nhất ('MaSP').
6. Với mỗi sản phẩm, nếu có nhiều hơn một dòng dữ liệu, nó sẽ huấn luyện một mô hình hồi quy tuyến tính riêng biệt để dự đoán 'GiaBan' dựa trên 'SoLuongBan'. Giá dự đoán được tính dựa trên số lượng bán trung bình của sản phẩm đó.
7. Nếu chỉ có một dòng dữ liệu cho sản phẩm, giá dự đoán sẽ là giá bán thực tế của dòng đó.
8. Nếu không có dữ liệu cho sản phẩm, giá dự đoán sẽ là None.
9. In kết quả: In ra DataFrame chứa 'MaSP' và 'GiaBanDuDoan'.
10. Xuất file CSV: Lưu kết quả vào file gia_ban_du_kien_sau_4_thang.csv.

In [75]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings

# Bỏ qua UserWarning và FutureWarning
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

def xuat_file_csv(df, ten_file, index=False, encoding='utf-8'):
    """
    Hàm xuất DataFrame ra file CSV.

    Args:
        df (pd.DataFrame): DataFrame cần xuất.
        ten_file (str): Tên file CSV (ví dụ: 'output.csv').
        index (bool, optional): Có ghi index của DataFrame vào file CSV không. Mặc định là False.
        encoding (str, optional): Mã hóa của file CSV. Mặc định là 'utf-8'.
    """
    try:
        df.to_csv(ten_file, index=index, encoding=encoding)
        print(f"Đã xuất thành công file CSV: '{ten_file}'")
    except Exception as e:
        print(f"Đã xảy ra lỗi khi xuất file CSV: {e}")

# Tên file CSV
ten_file = 'DuLieuBanHang_csv.csv'

try:
    # Đọc dữ liệu từ file CSV
    df = pd.read_csv(ten_file)
    df.rename(columns={'SanPham': 'MaSP', 'SoLuongBan': 'SoLuongBan'}, inplace=True)

    # Các biến độc lập (features) có thể ảnh hưởng đến giá
    cac_cot_dau_vao = ['SoLuongBan']
    cot_muc_tieu = 'GiaBan'

    # Tạo DataFrame để lưu giá dự đoán cho từng sản phẩm
    df_gia_du_doan = pd.DataFrame(columns=['MaSP', 'GiaBanDuDoan'])

    # Lặp qua từng sản phẩm duy nhất
    for ma_sp in df['MaSP'].unique():
        df_sp = df[df['MaSP'] == ma_sp].dropna(subset=cac_cot_dau_vao + [cot_muc_tieu])

        if len(df_sp) > 1:
            X = df_sp[cac_cot_dau_vao]
            y = df_sp[cot_muc_tieu]

            model = LinearRegression()
            model.fit(X, y)

            so_luong_ban_tb = df_sp['SoLuongBan'].mean()
            gia_ban_du_doan = model.predict([[so_luong_ban_tb]])[0]

            df_gia_du_doan = pd.concat([df_gia_du_doan, pd.DataFrame([{'MaSP': ma_sp, 'GiaBanDuDoan': gia_ban_du_doan}])], ignore_index=True)

        elif not df_sp.empty:
            gia_ban_thuc_te = df_sp['GiaBan'].iloc[0]
            df_gia_du_doan = pd.concat([df_gia_du_doan, pd.DataFrame([{'MaSP': ma_sp, 'GiaBanDuDoan': gia_ban_thuc_te}])], ignore_index=True)
        else:
            df_gia_du_doan = pd.concat([df_gia_du_doan, pd.DataFrame([{'MaSP': ma_sp, 'GiaBanDuDoan': None}])], ignore_index=True)

    print("\nGiá Bán Dự Đoán (dựa trên hồi quy tuyến tính theo từng sản phẩm):")
    print(df_gia_du_doan)

    # Xuất DataFrame df_gia_du_doan ra file CSV
    ten_file_xuat = 'gia_ban_du_kien_sau_4_thang.csv'
    xuat_file_csv(df_gia_du_doan, ten_file_xuat)

except FileNotFoundError:
    print(f"Không tìm thấy file: {ten_file}. Vui lòng kiểm tra lại tên file và đường dẫn.")
except Exception as e:
    print(f"Đã xảy ra lỗi: {e}")


Giá Bán Dự Đoán (dựa trên hồi quy tuyến tính theo từng sản phẩm):
    MaSP  GiaBanDuDoan
0   SP01     21.933333
1   SP02     18.083333
2   SP03     24.166667
3   SP04     20.933333
4   SP05     24.933333
5   SP06     23.850000
6   SP07     23.700000
7   SP08     17.683333
8   SP09     18.150000
9   SP10     18.450000
10  SP11     20.683333
11  SP12     17.533333
12  SP13     23.066667
13  SP14     22.766667
14  SP15     23.066667
15  SP16     23.150000
16  SP17     22.300000
17  SP18     20.550000
Đã xuất thành công file CSV: 'gia_ban_du_kien_sau_4_thang.csv'


In [ ]:
# Đưa ra đánh giá mô hình ---- Nháp ----
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings

# Bỏ qua UserWarning và FutureWarning
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

def xuat_file_csv(df, ten_file, index=False, encoding='utf-8'):
    """
    Hàm xuất DataFrame ra file CSV.

    Args:
        df (pd.DataFrame): DataFrame cần xuất.
        ten_file (str): Tên file CSV (ví dụ: 'output.csv').
        index (bool, optional): Có ghi index của DataFrame vào file CSV không. Mặc định là False.
        encoding (str, optional): Mã hóa của file CSV. Mặc định là 'utf-8'.
    """
    try:
        df.to_csv(ten_file, index=index, encoding=encoding)
        print(f"Đã xuất thành công file CSV: '{ten_file}'")
    except Exception as e:
        print(f"Đã xảy ra lỗi khi xuất file CSV: {e}")

def danh_gia_mo_hinh(y_true, y_pred, ma_sp):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    print(f"\nĐánh giá mô hình cho sản phẩm {ma_sp}:")
    print(f"  Mean Squared Error (MSE): {mse:.2f}")
    print(f"  R-squared (R^2): {r2:.2f}")
    print(f"  Mean Absolute Error (MAE): {mae:.2f}")
    if r2 > 0.7:
        print("  Mô hình có vẻ phù hợp tốt.")
    elif r2 > 0.5:
        print("  Mô hình ở mức chấp nhận được.")
    else:
        print("  Mô hình có thể chưa phù hợp tốt.")

# Tên file CSV
ten_file = 'DuLieuBanHang_csv.csv'

try:
    # Đọc dữ liệu từ file CSV
    df = pd.read_csv(ten_file)
    df.rename(columns={'SanPham': 'MaSP', 'SoLuongBan': 'SoLuongBan'}, inplace=True)

    # Các biến độc lập (features) có thể ảnh hưởng đến giá
    cac_cot_dau_vao = ['SoLuongBan']
    cot_muc_tieu = 'GiaBan'

    # Tạo DataFrame để lưu giá dự đoán cho từng sản phẩm
    df_gia_du_doan = pd.DataFrame(columns=['MaSP', 'GiaBanDuDoan'])

    # Lặp qua từng sản phẩm duy nhất
    for ma_sp in df['MaSP'].unique():
        df_sp = df[df['MaSP'] == ma_sp].dropna(subset=cac_cot_dau_vao + [cot_muc_tieu])

        if len(df_sp) > 5:  # Cần ít nhất vài điểm dữ liệu để chia train/test có ý nghĩa
            X = df_sp[cac_cot_dau_vao]
            y = df_sp[cot_muc_tieu]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            model = LinearRegression()
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            danh_gia_mo_hinh(y_test, y_pred, ma_sp)

            so_luong_ban_tb = df_sp['SoLuongBan'].mean()
            gia_ban_du_doan = model.predict([[so_luong_ban_tb]])[0]
            df_gia_du_doan = pd.concat([df_gia_du_doan, pd.DataFrame([{'MaSP': ma_sp, 'GiaBanDuDoan': gia_ban_du_doan}])], ignore_index=True)

        elif not df_sp.empty:
            gia_ban_thuc_te = df_sp['GiaBan'].iloc[0]
            df_gia_du_doan = pd.concat([df_gia_du_doan, pd.DataFrame([{'MaSP': ma_sp, 'GiaBanDuDoan': gia_ban_thuc_te}])], ignore_index=True)
        else:
            df_gia_du_doan = pd.concat([df_gia_du_doan, pd.DataFrame([{'MaSP': ma_sp, 'GiaBanDuDoan': None}])], ignore_index=True)

    print("\nGiá Bán Dự Đoán (dựa trên hồi quy tuyến tính theo từng sản phẩm):")
    print(df_gia_du_doan)

    # Xuất DataFrame df_gia_du_doan ra file CSV
    ten_file_xuat = 'gia_ban_du_kien_sau_4_thang.csv'
    xuat_file_csv(df_gia_du_doan, ten_file_xuat)

except FileNotFoundError:
    print(f"Không tìm thấy file: {ten_file}. Vui lòng kiểm tra lại tên file và đường dẫn.")
except Exception as e:
    print(f"Đã xảy ra lỗi: {e}")

In [ ]:
# ================= Đưa ra mức % khuyến mãi vào các ngày Lễ ================

# đọc dữ liệu từ file KyNghi.csv

# sử dụng mô hình hồi quy tuyến tính để dự đoán "% khuyến mãi" cho các ngày lễ có trong file CSV


In [76]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Đọc file CSV
try:
    df = pd.read_csv('KyNghi.csv')
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file KyNghi.csv. Vui lòng đảm bảo file tồn tại trong thư mục làm việc.")
    exit()

# Hàm tạo cột 'Phần trăm khuyến mãi' dựa trên quy tắc cho ngày lễ
def get_promotion(row):
    if pd.notnull(row['Ngay le']) and row['Ngay le'] != 'NULL':
        promotion = (
            row['Cuoi tuan'] * 2 +  # Thêm % nếu cuối tuần
            row['Nghi hoc'] * 3 +    # Thêm % nếu nghỉ học
            (row['Nhiet do TB'] - 25) * 0.3 + # Thêm % theo nhiệt độ (trên 25)
            row['Ngoai troi'] * 5      # Thêm % theo hoạt động ngoài trời
        )
        return max(0, promotion) # Đảm bảo khuyến mãi không âm
    else:
        return 0.0

df['Phần trăm khuyến mãi'] = df.apply(get_promotion, axis=1)

# Chuyển đổi biến 'Ngay le' thành biến số (cho mô hình)
df['La_Ngay_Le'] = df['Ngay le'].apply(lambda x: 1 if pd.notnull(x) and x != 'NULL' else 0)

# Chọn các biến độc lập và biến mục tiêu
features = ['Cuoi tuan', 'Nghi hoc', 'Nhiet do TB', 'Ngoai troi', 'La_Ngay_Le']
target = 'Phần trăm khuyến mãi'

# Loại bỏ các dòng có giá trị NaN trong các cột liên quan
df_cleaned = df.dropna(subset=features + [target])

X = df_cleaned[features]
y = df_cleaned[target]

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Khởi tạo và huấn luyện mô hình hồi quy tuyến tính trên TOÀN BỘ dữ liệu huấn luyện
model = LinearRegression()
model.fit(X_train, y_train)

# Dự đoán phần trăm khuyến mãi trên tập kiểm tra
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'\nMean Squared Error trên tập kiểm tra: {mse}')

# Đánh giá mô hình (dựa trên MSE)
print("\nĐánh giá mô hình (dựa trên MSE):")
if mse < 2:
    print("Mô hình có vẻ đạt chuẩn tốt.")
elif mse < 5:
    print("Mô hình ở mức chấp nhận được.")
else:
    print("Mô hình có thể chưa đạt chuẩn.")

# Lọc ra kết quả dự đoán chỉ cho các ngày lễ (Ngay le khác NULL) trên tập DỮ LIỆU GỐC đã làm sạch
df_le_results = df_cleaned[df_cleaned['Ngay le'].notna() & (df_cleaned['Ngay le'] != 'NULL')].copy()
df_le_results['Phần trăm khuyến mãi dự đoán'] = model.predict(df_le_results[features])

print("\nKết quả dự đoán phần trăm khuyến mãi cho các ngày lễ:")
print(df_le_results[['Ngay Thang', 'Ngay le', 'Phần trăm khuyến mãi', 'Phần trăm khuyến mãi dự đoán']])

# Xuất kết quả dự đoán cho ngày lễ ra file CSV
output_file = 'khuyen_mai_ngay_le.csv'
df_le_results[['Ngay Thang', 'Ngay le', 'Phần trăm khuyến mãi dự đoán']].to_csv(output_file, index=False)

print(f"\nĐã xuất kết quả dự đoán khuyến mãi cho các ngày lễ ra file: {output_file}")

print("\nCác hệ số của mô hình hồi quy (huấn luyện trên toàn bộ dữ liệu):")
coefficients = pd.DataFrame({'Feature': features, 'Coefficient': model.coef_})
print(coefficients)
print(f"\nIntercept: {model.intercept_}")


Mean Squared Error trên tập kiểm tra: 0.2443056088221271

Đánh giá mô hình (dựa trên MSE):
Mô hình có vẻ đạt chuẩn tốt.

Kết quả dự đoán phần trăm khuyến mãi cho các ngày lễ:
     Ngay Thang                        Ngay le  Phần trăm khuyến mãi  \
0    2023-01-01                 Tet duong lich                   8.3   
19   2023-01-20                 Tet Nguyen Dan                   6.3   
20   2023-01-21                 Tet Nguyen Dan                   8.3   
21   2023-01-22                 Tet Nguyen Dan                   8.3   
22   2023-01-23                 Tet Nguyen Dan                   6.3   
23   2023-01-24                 Tet Nguyen Dan                   6.3   
66   2023-03-08                 Quoc te Phu nu                   3.3   
107  2023-04-18              Gio to Hung Vuong                   8.3   
118  2023-04-29  Giai phong mien Nam (nghi bu)                   8.3   
119  2023-04-30            Giai phong mien Nam                   8.3   
120  2023-05-01               Qu

In [ ]:
Cột "Phần trăm khuyến mãi dự đoán" chứa kết quả dự đoán mức phần trăm khuyến mãi cho từng ngày
(bao gồm cả ngày lễ và ngày thường) mà mô hình hồi quy tuyến tính đã đưa ra sau khi được huấn luyện.

Mô hình này đã cố gắng học mối quan hệ giữa các features (Cuoi tuan, Nghi hoc, Nhiet do TB, Ngoai troi, La_Ngay_Le)
và target (cột "Phần trăm khuyến mãi" mà chúng ta đã tạo dựa trên quy tắc cho ngày lễ).

In [ ]:
# ======= CHỐT GIÁ BÁN PHÙ HỢP NHẤT CHO SP ========
# 1. Đọc dữ liệu từ cả ba file CSV vào các DataFrame.
# 2. Gộp các DataFrame này lại với nhau dựa trên cột 'MaSP'.
# 3. Chọn các cột giá dự đoán làm biến độc lập (features) và một trong số chúng (ví dụ, giá từ gia_ban_du_kien.csv) làm biến mục tiêu ban đầu để huấn luyện mô hình.
# 4. Huấn luyện mô hình hồi quy tuyến tính.
# 5. Sử dụng mô hình đã huấn luyện để dự đoán một mức giá mới, có thể dựa trên chính các features đó.

In [77]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

# 1. Đọc dữ liệu từ các file CSV
try:
    df_gia_ban = pd.read_csv('gia_ban_du_kien.csv')
    df_canh_tranh = pd.read_csv('gia_ban_du_kien_tu_CH_canh_tranh.csv')
    df_4_thang = pd.read_csv('gia_ban_du_kien_sau_4_thang.csv')
except FileNotFoundError as e:
    print(f"Không tìm thấy một hoặc nhiều file CSV: {e}")
    exit()

# Đổi tên cột để dễ quản lý
df_canh_tranh = df_canh_tranh.rename(columns={'GiaBanPhuHop': 'GiaBan_CanhTranh'})
df_4_thang = df_4_thang.rename(columns={'GiaBanDuDoan': 'GiaBan_4Thang'})
df_gia_ban = df_gia_ban.rename(columns={'Gia Ban Du Kien': 'GiaBan_ChiPhi'})

# 2. Gộp các DataFrame lại với nhau dựa trên cột 'MaSP'
df_merged = pd.merge(df_gia_ban[['MaSP', 'GiaBan_ChiPhi']], df_canh_tranh[['MaSP', 'GiaBan_CanhTranh']], on='MaSP', how='outer')
df_merged = pd.merge(df_merged, df_4_thang[['MaSP', 'GiaBan_4Thang']], on='MaSP', how='outer')

# 3. Chọn tất cả các cột giá dự đoán làm features
features = ['GiaBan_ChiPhi', 'GiaBan_CanhTranh', 'GiaBan_4Thang']

# Tạo một cột mục tiêu giả định (ví dụ, trung bình của các giá) để huấn luyện
df_merged['GiaTriTrungBinh'] = df_merged[features].mean(axis=1)
target = 'GiaTriTrungBinh'

# Loại bỏ các hàng chứa NaN ở các cột features và target
df_final = df_merged.dropna(subset=features + [target])

# Chuẩn bị dữ liệu cho mô hình
X = df_final[features]
y = df_final[target]

# 4. Huấn luyện mô hình hồi quy tuyến tính
model = LinearRegression()
model.fit(X, y)

# 5. Sử dụng mô hình đã huấn luyện để dự đoán mức giá kết hợp
gia_ban_du_doan_cuoi = model.predict(X)

# Tạo DataFrame kết quả
df_result = pd.DataFrame({'MaSP': df_final['MaSP'], 'GiaBanDuDoan': gia_ban_du_doan_cuoi})

# In kết quả
print("\nGiá bán dự đoán cuối cùng (dựa trên mô hình hồi quy từ tất cả các giá):")
print(df_result)

# Tạo nội dung file SQL
sql_commands = "-- Bảng chứa giá bán dự đoán\n"
sql_commands += "CREATE TABLE IF NOT EXISTS gia_du_doan (\n"
sql_commands += "    MaSP TEXT PRIMARY KEY,\n"
sql_commands += "    GiaBanDuDoan REAL\n"
sql_commands += ");\n\n"

sql_commands += "-- Xóa dữ liệu cũ (nếu cần)\n"
sql_commands += "DELETE FROM gia_du_doan;\n\n"

sql_commands += "-- Thêm dữ liệu giá bán dự đoán\n"
for index, row in df_result.iterrows():
    sql_commands += f"INSERT INTO gia_du_doan (MaSP, GiaBanDuDoan) VALUES ('{row['MaSP']}', {row['GiaBanDuDoan']});\n"

# Lưu nội dung vào file .sql
ten_file_sql = 'gia_ban_du_doan.sql'
with open(ten_file_sql, 'w') as f:
    f.write(sql_commands)

print(f"\nĐã lưu giá bán dự đoán vào file SQL: '{ten_file_sql}'")


Giá bán dự đoán cuối cùng (dựa trên mô hình hồi quy từ tất cả các giá):
    MaSP  GiaBanDuDoan
0   SP01     21.948539
1   SP02     17.356573
2   SP03     25.388889
3   SP04     25.644444
4   SP05     24.311111
5   SP06     23.803872
6   SP07     23.900000
7   SP08     15.561111
8   SP09     16.524169
9   SP10     15.483333
10  SP11     17.561111
11  SP12     14.844444
12  SP13     20.688889
13  SP14     19.922222
14  SP15     20.022222
15  SP16     21.050000
16  SP17     19.766667
17  SP18     22.183333

Đã lưu giá bán dự đoán vào file SQL: 'gia_ban_du_doan.sql'
